In [2]:
import xml.sax
import csv
import os

In [3]:
if os.getcwd().endswith('notebook'):
    os.chdir('../')
print(f'current working directory: {os.getcwd()}')

current working directory: /Users/howechen/Project/ntu_sd6103_team_project/ntu_sd6103_data_systems_team_project


In [17]:
class Find_K_Records(xml.sax.ContentHandler):
    def __init__(self, target_type, k):
        super().__init__()
        self.current_element = ""
        self.current_data = {}
        self.record_count = 0  # 用于计数已找到的记录
        self.found_target = False  # 标记是否找到目标记录
        self.target_type = target_type

    def startElement(self, tag, attributes):
        self.current_element = tag
        # 如果找到incollection类型的记录，则初始化记录数据
        if tag == self.target_type:
            self.current_data = {"type": tag, "id": attributes["key"], "date": attributes["mdate"]}
            self.found_target = True

    def endElement(self, tag):
        # 在incollection的记录结束时打印该记录的所有元素
        if tag == self.target_type and self.found_target:
            self.record_count += 1
            print(f"Record {self.record_count}: {self.current_data}")
            self.found_target = False  # 重置标志

            # 如果只需要查看一条记录，这里可以终止解析
            if self.record_count == k:
                raise xml.sax.SAXException("Found target, stop parsing")  # 通过抛出异常停止解析

        # 在找到incollection的记录时，将当前元素的内容添加到记录中
        if self.found_target and self.current_element:
            self.current_data[self.current_element] = self.content

    def characters(self, content):
        # 记录当前元素的内容
        self.content = content.strip()

# 初始化解析器和处理器
parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
target_type = "proceedings"
k = 5
handler = Find_K_Records(target_type=target_type, k=k)
parser.setContentHandler(handler)

# 解析XML文件
xml_file = "../ntu_sd6103_team_project_data/dblp.xml"
try:
    parser.parse(xml_file)
except xml.sax.SAXException:
    print(f"Found and printed {k} records of '{target_type}'")

Record 1: {'type': 'proceedings', 'id': 'reference/med/2013', 'date': '2017-05-16', 'editor': 'Ankur Agarwal', 'title': 'Handbook of Medical and Healthcare Technologies', 'year': '2013', 'publisher': 'Springer', 'ee': 'https://doi.org/10.1007/978-1-4614-8495-0', 'isbn': '978-1-4614-8494-3', 'url': 'db/reference/med/med2013.html'}
Record 2: {'type': 'proceedings', 'id': 'reference/genetic/2015', 'date': '2020-03-27', 'editor': 'Conor Ryan', 'title': 'Handbook of Genetic Programming Applications', 'publisher': 'Springer', 'year': '2015', 'isbn': '978-3-319-20882-4', 'ee': 'https://doi.org/10.1007/978-3-319-20883-1', 'url': 'db/reference/genetic/genetic2015.html'}
Record 3: {'type': 'proceedings', 'id': 'series/isrl/97', 'date': '2017-05-16', 'editor': 'Seda Yanik', 'title': 'Intelligent Decision Making in Quality Management - Theory and Applications', 'publisher': 'Springer', 'year': '2016', 'series': 'Intelligent Systems Reference Library', 'volume': '97', 'isbn': '978-3-319-24497-6', '

## Classes

**Incollection record:**
- type: incollection (no null)
- id: one and only (no null)
- date: date (no null)
- author
- title: incollection title
- year: year
- booktitle:
- ee
- url

In [22]:
class incollectionParser(xml.sax.ContentHandler):
    def __init__(self, batch_size=5000):
        super().__init__()
        self.current_element = ""
        self.current_data = {}
        self.records = []
        self.authors = []
        self.orcid = []
        self.batch_size = batch_size
        self.file_count = 1

    def startElement(self, tag, attributes):
        self.current_element = tag
        if tag == 'incollection':
            # Reset for each new publication record
            # print(self.current_data)
            # print(f"*****Processing {tag}...******")
            self.current_data = {
                "type": tag,
                "id": attributes["key"],
                "date": attributes["mdate"],
                "title": "",
                "pages": "",
                "year": "",
                "booktitle": "",
                "publisher": "",
                "ee": "",
                "url": ""
            }
            self.authors = []

    def endElement(self, tag):
        if tag in ["incollection"]:
            # Add authors as a comma-separated string
            self.current_data["authors"] = ", ".join(self.authors)
            self.records.append(self.current_data)
            if len(self.records) >= self.batch_size:
                self.save_to_csv()
                self.records = []  # Reset records
        elif tag == "author":
            # Append author to authors list
            self.authors.append(self.content)
            # print(f"author: {self.content}")
        elif tag in self.current_data:
            # Save the content to the current field
            self.current_data[tag] = self.content
            # print(f"{tag}: {self.content}")

    def characters(self, content):
        self.content = content.strip()

    def save_to_csv(self):
        if not os.path.exists('../ntu_sd6103_team_project_data'):
            os.mkdir('../ntu_sd6103_team_project_data')
        if not os.path.exists('../ntu_sd6103_team_project_data/csv_files'):
            os.mkdir('../ntu_sd6103_team_project_data/csv_files')
        if not os.path.exists('../ntu_sd6103_team_project_data/csv_files/incollection'):
            os.mkdir('../ntu_sd6103_team_project_data/csv_files/incollection')
        csv_file = f"../ntu_sd6103_team_project_data/csv_files/incollection/dplr_incollection_part_{self.file_count}.csv"
        with open(csv_file, "w", newline='', encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=self.records[0].keys())
            writer.writeheader()
            writer.writerows(self.records)
        print(f"Saving batch {self.file_count} to {csv_file}")
        self.file_count += 1

phdthesis record:

- type: phdthesis
- id: 
- date
- author
- title
- publisher
- year
- series
- volume
- pages
- school
- isbn
- ee
- note

In [19]:
class phdthesisParser(xml.sax.ContentHandler):
    def __init__(self, batch_size=5000):
        super().__init__()
        self.current_element = ""
        self.current_data = {}
        self.records = []
        self.authors = []
        self.batch_size = batch_size
        self.file_count = 1

    def startElement(self, tag, attributes):
        self.current_element = tag
        if tag == 'phdthesis':
            # Reset for each new publication record
            # print(self.current_data)
            # print(f"*****Processing {tag}...******")
            self.current_data = {
                "type": tag,
                "id": attributes["key"],
                "date": attributes["mdate"],
                "title": "",
                "pages": "",
                "publisher": "",
                "year": "",
                "series": "",
                "volume": "",
                "school": "",
                "isbn": "",
                "ee": ""
            }
            self.authors = []

    def endElement(self, tag):
        if tag in ["phdthesis"]:
            # Add authors as a comma-separated string
            self.current_data["authors"] = ", ".join(self.authors)
            self.records.append(self.current_data)
            if len(self.records) >= self.batch_size:
                self.save_to_csv()
                self.records = []  # Reset records
        elif tag == "author":
            # Append author to authors list
            self.authors.append(self.content)
            # print(f"author: {self.content}")
        elif tag in self.current_data:
            # Save the content to the current field
            self.current_data[tag] = self.content
            # print(f"{tag}: {self.content}")

    def characters(self, content):
        self.content = content.strip()

    def save_to_csv(self):
        if not os.path.exists('../ntu_sd6103_team_project_data'):
            os.mkdir('../ntu_sd6103_team_project_data')
        if not os.path.exists('../ntu_sd6103_team_project_data/csv_files'):
            os.mkdir('../ntu_sd6103_team_project_data/csv_files')
        if not os.path.exists('../ntu_sd6103_team_project_data/csv_files/phdthesis'):
            os.mkdir('../ntu_sd6103_team_project_data/csv_files/phdthesis')
        csv_file = f"../ntu_sd6103_team_project_data/csv_files/phdthesis/dplr_phdthesis_part_{self.file_count}.csv"
        with open(csv_file, "w", newline='', encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=self.records[0].keys())
            writer.writeheader()
            writer.writerows(self.records)
        print(f"Saving batch {self.file_count} to {csv_file}")
        self.file_count += 1

In [24]:
class masterthesisParser(xml.sax.ContentHandler):
    def __init__(self, batch_size=5000):
        super().__init__()
        self.current_element = ""
        self.current_data = {}
        self.records = []
        self.authors = []
        self.batch_size = batch_size
        self.file_count = 1

    def startElement(self, tag, attributes):
        self.current_element = tag
        if tag == 'masterthesis':
            # Reset for each new publication record
            # print(self.current_data)
            print(f"*****Processing {tag}...******")
            self.current_data = {
                "type": tag,
                "id": attributes["key"],
                "date": attributes["mdate"],
                "title": "",
                "year": "",
                "school": "",
                "ee": ""
            }
            self.authors = []

    def endElement(self, tag):
        if tag in ["masterthesis"]:
            # Add authors as a comma-separated string
            self.current_data["authors"] = ", ".join(self.authors)
            self.records.append(self.current_data)
            if len(self.records) >= self.batch_size:
                self.save_to_csv()
                self.records = []  # Reset records
        elif tag == "author":
            # Append author to authors list
            self.authors.append(self.content)
            # print(f"author: {self.content}")
        elif tag in self.current_data:
            # Save the content to the current field
            self.current_data[tag] = self.content
            # print(f"{tag}: {self.content}")

    def characters(self, content):
        self.content = content.strip()

    def save_to_csv(self):
        if not os.path.exists('../ntu_sd6103_team_project_data'):
            os.mkdir('../ntu_sd6103_team_project_data')
        if not os.path.exists('../ntu_sd6103_team_project_data/csv_files'):
            os.mkdir('../ntu_sd6103_team_project_data/csv_files')
        if not os.path.exists('../ntu_sd6103_team_project_data/csv_files/masterthesis'):
            os.mkdir('../ntu_sd6103_team_project_data/csv_files/masterthesis')
        csv_file = f"../ntu_sd6103_team_project_data/csv_files/masterthesis/dplr_masterthesis_part_{self.file_count}.csv"
        with open(csv_file, "w", newline='', encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=self.records[0].keys())
            writer.writeheader()
            writer.writerows(self.records)
        print(f"Saving batch {self.file_count} to {csv_file}")
        self.file_count += 1

www records:

- type
- id
- date
- title
- pages
- note
- url
- authors

In [25]:
class wwwParser(xml.sax.ContentHandler):
    def __init__(self, batch_size=5000):
        super().__init__()
        self.current_element = ""
        self.current_data = {}
        self.records = []
        self.authors = []
        self.batch_size = batch_size  # batch size
        self.file_count = 1  # csv No.

    def startElement(self, tag, attributes):
        self.current_element = tag
        if tag == 'www':
            # Reset for each new publication record
            # print(self.current_data)
            # print(f"*****Processing {tag}...******")
            self.current_data = {
                "type": tag,
                "id": attributes["key"],
                "date": attributes["mdate"],
                "title": "",
                "pages": "",
                "note": "",
                "url": ""
            }
            self.authors = []

    def endElement(self, tag):
        if tag in ["www"]:
            # Add authors as a comma-separated string
            self.current_data["authors"] = ", ".join(self.authors)
            self.records.append(self.current_data)
            if len(self.records) >= self.batch_size:
                self.save_to_csv()
                self.records = []  # Reset records
        elif tag == "author":
            # Append author to authors list
            self.authors.append(self.content)
            # print(f"author: {self.content}")
        elif tag in self.current_data:
            # Save the content to the current field
            self.current_data[tag] = self.content
            # print(f"{tag}: {self.content}")

    def characters(self, content):
        self.content = content.strip()
    
    def save_to_csv(self):
        if not os.path.exists('../ntu_sd6103_team_project_data'):
            os.mkdir('../ntu_sd6103_team_project_data')
        if not os.path.exists('../ntu_sd6103_team_project_data/csv_files'):
            os.mkdir('../ntu_sd6103_team_project_data/csv_files')
        if not os.path.exists('../ntu_sd6103_team_project_data/csv_files/www'):
            os.mkdir('../ntu_sd6103_team_project_data/csv_files/www')
        csv_file = f"../ntu_sd6103_team_project_data/csv_files/www/dplr_www_part_{self.file_count}.csv"
        with open(csv_file, "w", newline='', encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=self.records[0].keys())
            writer.writeheader()
            writer.writerows(self.records)
        print(f"Saving batch {self.file_count} to {csv_file}")
        self.file_count += 1

data record:

- type,
- id,
- date,
- title,
- pages,
- publisher,
- year,
- month,
- ee,
- stream,
- rel,
- authors

In [26]:
class dataParser(xml.sax.ContentHandler):
    def __init__(self, batch_size=5000):
        super().__init__()
        self.current_element = ""
        self.current_data = {}
        self.records = []
        self.authors = []
        self.batch_size = batch_size  # batch size
        self.file_count = 1  # csv No.

    def startElement(self, tag, attributes):
        self.current_element = tag
        if tag == 'data':
            # Reset for each new publication record
            # print(self.current_data)
            # print(f"*****Processing {tag}...******")
            self.current_data = {
                "type": tag,
                "id": attributes["key"],
                "date": attributes["mdate"],
                "title": "",
                "pages": "",
                "publisher": "",
                "year": "",
                "month": "",
                "ee": "",
                "publisher": "",
                "stream": "",
                "rel": ""
            }
            self.authors = []

    def endElement(self, tag):
        if tag in ["data"]:
            # Add authors as a comma-separated string
            self.current_data["authors"] = ", ".join(self.authors)
            self.records.append(self.current_data)
            if len(self.records) >= self.batch_size:
                self.save_to_csv()
                self.records = []  # Reset records
        elif tag == "author":
            # Append author to authors list
            self.authors.append(self.content)
            # print(f"author: {self.content}")
        elif tag in self.current_data:
            # Save the content to the current field
            self.current_data[tag] = self.content
            # print(f"{tag}: {self.content}")

    def characters(self, content):
        self.content = content.strip()
    
    def save_to_csv(self):
        if not os.path.exists('../ntu_sd6103_team_project_data'):
            os.mkdir('../ntu_sd6103_team_project_data')
        if not os.path.exists('../ntu_sd6103_team_project_data/csv_files'):
            os.mkdir('../ntu_sd6103_team_project_data/csv_files')
        if not os.path.exists('../ntu_sd6103_team_project_data/csv_files/data'):
            os.mkdir('../ntu_sd6103_team_project_data/csv_files/data')
        csv_file = f"../ntu_sd6103_team_project_data/csv_files/data/dplr_data_part_{self.file_count}.csv"
        with open(csv_file, "w", newline='', encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=self.records[0].keys())
            writer.writeheader()
            writer.writerows(self.records)
        print(f"Saving batch {self.file_count} to {csv_file}")
        self.file_count += 1

## Running

In [23]:
# Initialize the parser and handler
parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
handler = incollectionParser(batch_size=5000000)
parser.setContentHandler(handler)

# Parse the XML file
xml_file = "../ntu_sd6103_team_project_data/dblp.xml"
parser.parse(xml_file)

if handler.records:
    print("Saving the last batch...")
    handler.save_to_csv()   # Save the last batch

Saving the last batch...
Saving batch 1 to ../ntu_sd6103_team_project_data/csv_files/incollection/dplr_incollection_part_1.csv


In [20]:
# Initialize the parser and handler
parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
handler = phdthesisParser(batch_size=50000000)
parser.setContentHandler(handler)

# Parse the XML file
xml_file = "../ntu_sd6103_team_project_data/dblp.xml"
parser.parse(xml_file)

if handler.records:
    print("Saving the last batch...")
    handler.save_to_csv()   # Save the last batch

Saving the last batch...
Saving batch 1 to ../ntu_sd6103_team_project_data/csv_files/phdthesis/dplr_phdthesis_part_1.csv


In [27]:
# Initialize the parser and handler
parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
handler = masterthesisParser(batch_size=500000)
parser.setContentHandler(handler)

# Parse the XML file
xml_file = "../ntu_sd6103_team_project_data/dblp.xml"
parser.parse(xml_file)

if handler.records:
    print("Saving the last batch...")
    handler.save_to_csv()   # Save the last batch

In [28]:
# Initialize the parser and handler
parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
handler = wwwParser(batch_size=500000)
parser.setContentHandler(handler)

# Parse the XML file
xml_file = "../ntu_sd6103_team_project_data/dblp.xml"
parser.parse(xml_file)

if handler.records:
    print("Saving the last batch...")
    handler.save_to_csv()   # Save the last batch

Saving batch 1 to ../ntu_sd6103_team_project_data/csv_files/www/dplr_www_part_1.csv
Saving batch 2 to ../ntu_sd6103_team_project_data/csv_files/www/dplr_www_part_2.csv
Saving batch 3 to ../ntu_sd6103_team_project_data/csv_files/www/dplr_www_part_3.csv
Saving batch 4 to ../ntu_sd6103_team_project_data/csv_files/www/dplr_www_part_4.csv
Saving batch 5 to ../ntu_sd6103_team_project_data/csv_files/www/dplr_www_part_5.csv
Saving batch 6 to ../ntu_sd6103_team_project_data/csv_files/www/dplr_www_part_6.csv
Saving batch 7 to ../ntu_sd6103_team_project_data/csv_files/www/dplr_www_part_7.csv
Saving the last batch...
Saving batch 8 to ../ntu_sd6103_team_project_data/csv_files/www/dplr_www_part_8.csv


In [29]:
# Initialize the parser and handler
parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
handler = dataParser(batch_size=500000)
parser.setContentHandler(handler)

# Parse the XML file
xml_file = "../ntu_sd6103_team_project_data/dblp.xml"
parser.parse(xml_file)

if handler.records:
    print("Saving the last batch...")
    handler.save_to_csv()   # Save the last batch

Saving the last batch...
Saving batch 1 to ../ntu_sd6103_team_project_data/csv_files/data/dplr_data_part_1.csv
